In [ ]:
def load_trade_multiprocessing(filename,
               tz_exchange="America/New_York",
               only_non_special_trades=True,
               only_regular_trading_hours=True,
               open_time="09:30:00",
               close_time="16:00:00",
               merge_sub_trades=True):
    import pandas as pd
    DF = pd.read_csv(filename)
    if DF.empty:
        return None
    if only_non_special_trades:
        DF = DF[DF["trade-stringflag"] == "uncategorized"]
    DF.drop(columns=["trade-rawflag", "trade-stringflag"], axis=1, inplace=True)
    DF.index = pd.to_datetime(DF["xltime"], unit="D", origin="1899-12-30", utc=True)
    DF.index = DF.index.tz_convert(tz_exchange)
    DF.drop(columns="xltime", inplace=True)
    if only_regular_trading_hours:
        DF = DF.between_time(open_time, close_time)
    if merge_sub_trades:
        DF = DF.groupby(DF.index).agg(trade_price=pd.NamedAgg(column='trade-price', aggfunc='mean'),
                                      trade_volume=pd.NamedAgg(column='trade-volume', aggfunc='sum'))
    return DF


def load_bbo_multiprocessing(filename,
             tz_exchange="America/New_York",
             open_time="09:30:00",
             close_time="16:00:00",
             only_regular_trading_hours=True):
    import pandas as pd
    DF = pd.read_csv(filename)
    if DF.empty:
        return None
    DF.index = pd.to_datetime(DF["xltime"], unit="D", origin="1899-12-30", utc=True)
    DF.index = DF.index.tz_convert(tz_exchange)
    DF.drop(columns="xltime", inplace=True)
    if only_regular_trading_hours:
        DF = DF.between_time(open_time, close_time)
    return DF